# Travel and Tourism Reform Project

### Documentation

**Dataframes:** 
- df_qcontcust_2009_2019 -> contains data on all years between 2009 - 2019
- df_qcontcust_2009, df_qcontcust_2010, ... to df_qcontcust_2019 -> filtered from df_qcontcust_2009_2019 for each year
- df_qcontcust_2022 -> contains data for 2022 

**Dictionaries:**
- flow_dict -> contains flow codes (arrival/departure, foreign/UK) for all years
- Purpose_value_map_0919 -> Purpose of visit mapping for the years 2009 to 2019
- Purpose_value_map_22 -> Purpose of visit mapping for 2022
- Nationality_value_map_0919 -> mapping for Nationality of respondent - NEW CODES (2009-2019)
- Nationality_value_map_22 -> mapping for Nationality of respondent - NEW CODES (2022)

**New variables created:**

***


## Importing Packages

In [1]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json

import statsmodels.api as sm
from statsmodels.sandbox.stats.multicomp import multipletests

import scipy.stats as ss
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from scikit_posthocs import posthoc_dunn

from itertools import product

from imblearn.over_sampling import RandomOverSampler

from tabulate import tabulate

import warnings
warnings.filterwarnings("ignore")

## Loading Data

In [2]:
df_qcontcust_2009_2019 = pd.read_csv("C:\\Users\\medasud\\Downloads\\2013-UKDA-7380-tab\\tab\\qcontcust_2009_2019.tab", delimiter='\t')
#filtering the dataset into different years
df_qcontcust_2009 = df_qcontcust_2009_2019[df_qcontcust_2009_2019['Year'] == 2009]
df_qcontcust_2010 = df_qcontcust_2009_2019[df_qcontcust_2009_2019['Year'] == 2010]
df_qcontcust_2011 = df_qcontcust_2009_2019[df_qcontcust_2009_2019['Year'] == 2011]
df_qcontcust_2012 = df_qcontcust_2009_2019[df_qcontcust_2009_2019['Year'] == 2012]
df_qcontcust_2013 = df_qcontcust_2009_2019[df_qcontcust_2009_2019['Year'] == 2013]
df_qcontcust_2014 = df_qcontcust_2009_2019[df_qcontcust_2009_2019['Year'] == 2014]
df_qcontcust_2015 = df_qcontcust_2009_2019[df_qcontcust_2009_2019['Year'] == 2015]
df_qcontcust_2016 = df_qcontcust_2009_2019[df_qcontcust_2009_2019['Year'] == 2016]
df_qcontcust_2017 = df_qcontcust_2009_2019[df_qcontcust_2009_2019['Year'] == 2017]
df_qcontcust_2018 = df_qcontcust_2009_2019[df_qcontcust_2009_2019['Year'] == 2018]
df_qcontcust_2019 = df_qcontcust_2009_2019[df_qcontcust_2009_2019['Year'] == 2019]
df_qcontcust_2022 = pd.read_csv("C:\\Users\\medasud\\Downloads\\2022-UKDA-9122-tab\\tab\\qcontcust2022.tab", delimiter='\t')


df_qreg_2013 = pd.read_csv("C:\\Users\\medasud\\Downloads\\2013-UKDA-7380-tab\\tab\\qreg_2013.tab", delimiter='\t')
df_qreg_2014 = pd.read_csv("C:\\Users\\medasud\\Downloads\\2014-UKDA-7534-tab\\tab\\qreg_2014.tab", delimiter='\t')
df_qreg_2015 = pd.read_csv("C:\\Users\\medasud\\Downloads\\2015-UKDA-7754-tab\\tab\\qreg_2015.tab", delimiter='\t')
df_qreg_2016 = pd.read_csv("C:\\Users\\medasud\\Downloads\\2016-UKDA-8016-tab\\tab\\qreg_2016.tab", delimiter='\t')
df_qreg_2017 = pd.read_csv("C:\\Users\\medasud\\Downloads\\2017-UKDA-8286-tab\\tab\\qreg_2017.tab", delimiter='\t')
df_qreg_2018 = pd.read_csv("C:\\Users\\medasud\\Downloads\\2018-UKDA-8468-tab\\tab\\qreg_2018.tab", delimiter='\t')
df_qreg_2019 = pd.read_csv("C:\\Users\\medasud\\Downloads\\2019-UKDA-8575-tab\\tab\\qreg_2019.tab", delimiter='\t')
df_qreg_2022 = pd.read_csv("C:\\Users\\medasud\\Downloads\\2022-UKDA-9122-tab\\tab\\qreg_2022.tab", delimiter='\t')
#qreg is not available for 2009-2012


## Creating New Variables from Mappings

In [18]:
#dictionary for flow
flow_dict = {
    1.0: "Air Departure Foreign",
    2.0: "Air Departure UK",
    3.0: "Air Arrival Foreign",
    4.0: "Air Arrival UK",
    5.0: "Sea Departure Foreign",
    6.0: "Sea Departure UK",
    7.0: "Sea Arrival Foreign",
    8.0: "Sea Arrival UK"
}

#function to create Flow_Label column for all years

def create_flow_label_column(df):
    """
    This function creates a new column FLow_Label which is derived from the column Flow
    and an external data dictionary mapping the integer/float values in Flow to their respective 
    values. This is for all years.

    Parameters:
    param1 : the dataframe being manipulated
 
    Returns:
    int: no return value. When the function is called, the new column is created.
    """
    #fill missing values in Purpose column with -1
    df['Flow'].replace(' ', np.nan, inplace=True)
    df['Flow'].fillna(-1, inplace=True)
    df['Flow'] = df['Flow'].astype(float)
    df['Flow'].replace('-1', np.nan, inplace=True)
    
    df['Flow_Label'] = df['Flow'].map(flow_dict)

#call this function for df_qcontcust of each year
dataframes = [df_qcontcust_2009, df_qcontcust_2010, df_qcontcust_2011, df_qcontcust_2012,
              df_qcontcust_2013, df_qcontcust_2014, df_qcontcust_2015, df_qcontcust_2016,
              df_qcontcust_2017, df_qcontcust_2018, df_qcontcust_2019, df_qcontcust_2022]

#iterate over the list of dataframes and apply the function
for df in dataframes:
    create_flow_label_column(df)

In [19]:
#create new column Purpose_Label for years 2009-19

#load the mapping from the JSON file
file_path = "C:\\Users\\medasud\\Documents\\Project1\\Purpose_value_map_0919.json"
with open(file_path, 'r') as json_file:
    purpose_mapping_0919 = json.load(json_file)

#function to create Purpose_Label column
def create_purpose_column_0919(df, mapping):
    """
    This function creates a new column Purpose_Label which is derived from the column Purpose
    and an external data dictionary mapping the integer/float values in Purpose to their respective 
    purposes. This is for the years 2009-2019 only.

    Parameters:
    param1 : the dataframe being manipulated
    param2 : the mapping from the json file

    Returns:
    int: no return value. When the function is called, the new column is created.
    """
    df['Purpose'].replace(' ', pd.NA, inplace=True)
    df['Purpose'].fillna(-1, inplace=True)
    df['Purpose'] = df['Purpose'].astype(float)
    df['Purpose'] = df['Purpose'].astype(str)
    
    # Create a new column "Purpose_Label" by mapping the values
    df['Purpose_Label'] = df['Purpose'].map(mapping)
    df['Purpose'] = df['Purpose'].astype(float)
    
#call this function for df_qcontcust of each year
dataframes = [df_qcontcust_2009, df_qcontcust_2010, df_qcontcust_2011, df_qcontcust_2012,
              df_qcontcust_2013, df_qcontcust_2014, df_qcontcust_2015, df_qcontcust_2016,
              df_qcontcust_2017, df_qcontcust_2018, df_qcontcust_2019]

#iterate over the list of dataframes and apply the function
for df in dataframes:
    create_purpose_column_0919(df, purpose_mapping_0919)


In [20]:
#create new column Purpose_Label for years 2022

#load the mapping from the JSON file
file_path = "C:\\Users\\medasud\\Documents\\Project1\\Purpose_value_map_22.json"
with open(file_path, 'r') as json_file:
    purpose_mapping_22 = json.load(json_file)
    
#function to create Purpose_Label column
def create_purpose_column_22(df, mapping):
    """
    This function creates a new column Purpose_Label which is derived from the column Purpose
    and an external data dictionary mapping the integer/float values in Purpose to their respective 
    purposes. This is for the years 2022 only as the Purpose codes are different for this year.
    Purpose codes for subsequent years are likely to remain the same, and in that case, this function
    can be reused.

    Parameters:
    param1 : the dataframe being manipulated
    param2 : the mapping from the json file

    Returns:
    int: no return value. When the function is called, the new column is created.
    """
    
    df['Purpose'].replace(' ', pd.NA, inplace=True)
    df['Purpose'].fillna(-1, inplace=True)
    df['Purpose'] = df['Purpose'].astype(float)
    df['Purpose'] = df['Purpose'].astype(str)
    
    # Create a new column "Purpose_Label" by mapping the values
    df['Purpose_Label'] = df['Purpose'].map(mapping)
    df['Purpose'] = df['Purpose'].astype(float)
    
#call the function
create_purpose_column_22(df_qcontcust_2022, purpose_mapping_22)


In [21]:
df_qcontcust_2022['Purpose_Label'].value_counts()

Purpose_Label
Holiday/pleasure                                               51586
Visit family (priority)                                        29972
Business; Work                                                 11315
Visit friends                                                   4723
Same day transit                                                2970
Overnight transit                                               1372
OTHER                                                           1098
Watch sport                                                     1043
Play amateur sport                                               645
Definite job to go to                                            528
Medical Treatment                                                476
Cruise 0-2 nights ashore - For                                   242
International commuter                                           216
Military or embassy (serving on duty)                            178
First or Foundation 

In [ ]:
#function to load the mapping from the JSON file
file_path = "C:\\Users\\medasud\\Documents\\Project1\\Nationality_value_map_22.json"
def load_nationality_mapping_22(file_path):
    with open(file_path, 'r') as json_file:
        nationality_mapping_22 = json.load(json_file)
    return nationality_mapping_22


In [ ]:
#function to load the mapping from the JSON file
file_path = "C:\\Users\\medasud\\Documents\\Project1\\Nationality_value_map_0919.json"
def load_nationality_mapping(file_path):
    with open(file_path, 'r') as json_file:
        nationality_mapping = json.load(json_file)
    return nationality_mapping


In [23]:
#creating a mapping of the values in Purpose for 2022 (different from 2009-2019)

Purpose_value_map_22 = {
    10.0: "Holiday/pleasure",
    11.0: "Visit family (priority)",
    12.0: "Visit friends",
    13.0: "Getting married",
    14.0: "Play amateur sport",
    15.0: "Watch sport",
    16.0: "Personal shopping",
    17.0: "Cruise 0-2 nights ashore - UK",
    18.0: "Cruise 0-2 nights ashore - For",
    20.0: "Business; Work",
    21.0: "Visit trade fair",
    22.0: "Conference 20+ people",
    23.0: "Definite job to go to",
    24.0: "International commuter",
    25.0: "Looking for work",
    26.0: "Au Pair",
    27.0: "Working Holiday",
	30.0: "Olympics/Paralympics Participate",
	31.0: "Olympics/Paralympics Work",
	32.0: "Olympics/Paralympics Watch",
	41.0: "Medical Treatment",
	43.0: "Joining another traveller",
	44.0: "Accompany another traveller",
    45.0: "OTHER",
    46.0: "Religious Pilgrimage",
	47.0: "University Degree or Diploma",
	50.0: "Asylum Seeker",
    51.0: "Immigrating/Emigrating",
    52.0: "Returning Home To Live",
	60.0: "Formal Course",
	61.0: "First or Foundation Degree",
	62.0: "Higher or Postgraduate Degree",
    63.0: "English language course (not degree level)",
    64.0: "Other Course Below Degree Level & Above Secondary Education",
    65.0: "Secondary education",
    66.0: "Professional qualification",
    70.0: "Overnight transit",
    71.0: "Same day transit",
    80.0: "Military or embassy (serving on duty)",
    81.0: "Merchant navy (joining or leaving ship)",
    82.0: "Airline crew (positioning)",
    83.0: "Unacc schoolchild (16 or under, school to parents)",
    84.0: "Embassy Personel"
}

In [19]:
#function to create Purpose_Label column for 2009-2019

#function to load the mapping from the JSON file
file_path = "C:\\Users\\medasud\\Desktop\\DSDP_Project1\\Purpose_value_map_0919.json"
def load_purpose_mapping(file_path):
    with open(file_path, 'r') as json_file:
        purpose_mapping = json.load(json_file)
    return purpose_mapping

#load the purpose mapping from the JSON file
Purpose_value_map_0919 = load_purpose_mapping('Purpose_value_map_0919.json')


In [21]:
def create_purpose_label_column(df):
    df['Purpose'].replace(' ', np.nan, inplace=True)
    
    
    df['Purpose'].replace(' ', np.nan, inplace=True)
    # Fill missing values in "Purpose" column with a default value, for example, -1
    df['Purpose'].fillna(-1, inplace=True)
    # Convert "Purpose" column to float
    df['Purpose'] = df['Purpose'].astype(float)
    df['Purpose'].replace('-1', np.nan, inplace=True)
    df['Purpose'] = df['Purpose'].astype(str)
    df['Purpose'].replace('-1', "Unknown", inplace=True)
    
    # Create a new column "Purpose_Label" by mapping the values
    df['Purpose_Label'] = df['Purpose'].map(Purpose_value_map_0919)
  
#call this function for df_qcontcust of each year
dataframes = [df_qcontcust_2009, df_qcontcust_2010, df_qcontcust_2011, df_qcontcust_2012,
              df_qcontcust_2013, df_qcontcust_2014, df_qcontcust_2015, df_qcontcust_2016,
              df_qcontcust_2017, df_qcontcust_2018, df_qcontcust_2019]

#iterate over the list of dataframes and apply the function for 2009-2019
for df in dataframes:
    create_purpose_label_column(df)


In [22]:
df_qcontcust_2013['Purpose_Label'].value_counts()

Purpose_Label
Holiday/pleasure                                      50472
Visit family (priority)                               22423
Business; Work                                        18709
Visit friends                                          4677
Same day transit                                       3180
Overnight transit                                      1331
Play amateur sport                                     1157
OTHER                                                  1151
Watch sport                                            1060
Personal shopping                                       743
Other formal study                                      654
Cruise 0-2 nights ashore - For                          408
Definite job to go to                                   345
Accompany / join                                        242
Medical treatment                                       211
Cruise 0-2 nights ashore - UK                           209
Military (serving on duty)

In [20]:
#visit purposes that we're not interested in 

excluded_purposes_22 = ["International commuter", "Immigrating/Emigrating",  "Asylum Seeker", 
                        "Returning Home To Live", "Overnight transit", "Same day transit", 
                        "Military or embassy (serving on duty)", "Merchant navy (joining or leaving ship)", 
                        "Airline crew (positioning)", "Looking for work"]

In [8]:
#create the column for 2022

df_qcontcust_2022['Purpose'].replace(' ', np.nan, inplace=True)
df_qcontcust_2022['Purpose'] = pd.to_numeric(df_qcontcust_2022['Purpose'], errors='coerce')
#Purpose column was not numeric
df_qcontcust_2022['Purpose_Label'] = df_qcontcust_2022['Purpose'].map(Purpose_value_map_22)
df_qcontcust_2022['Purpose_Label'].fillna(("Unknown"), inplace=True)
df_qcontcust_2022['Purpose'].replace('np.nan', -1, inplace=True)


In [9]:
Nationality_value_map_0919 = {
    0.0: "Staff/military/embassy",
    4.0: "Afghanistan",
    8.0: "Albania",
    10.0: "Antarctica (Foreign)",
    12.0: "Algeria",
    16.0: "American Samoa/Oceania",
    20.0: "Andorra",
    24.0: "Angola",
    28.0: "Barbuda/Antigua",
    31.0: "Azerbaijan",
    32.0: "Argentina",
    36.0: "Australia",
    40.0: "Austria",
    44.0: "Bahamas",
    48.0: "Bahrain",
    50.0: "Bangladesh",
    51.0: "Armenia",
    52.0: "Barbados",
    56.0: "Belgium",
    60.0: "Bermuda",
    64.0: "Bhutan",
    68.0: "Bolivia",
    70.0: "Bosnia Herzegovina",
    72.0: "Botswana",
    74.0: "Bouvet Island",
    76.0: "Brazil",
    84.0: "Belize",
    86.0: "British Indian Ocean Territory",
    90.0: "Solomon Islands",
    92.0: "Virgin Islands (British)",
    96.0: "Brunei",
    100.0: "Bulgaria",
    104.0: "Myanmar (Burma)",
    108.0: "Burundi",
    112.0: "Belarus",
    116.0: "Cambodia",
    120.0: "Cameroon",
    124.0: "Canada",
    132.0: "Cape Verde Islands",
    136.0: "Cayman Islands",
    140.0: "Central African Republic",
    144.0: "Sri Lanka",
    148.0: "Chad",
    152.0: "Chile",
    156.0: "China/Tibet",
    158.0: "Taiwan",
    162.0: "Christmas Island/Oceania",
    166.0: "Cocos Island/Oceania",
    170.0: "Colombia",
    174.0: "Comoros",
    175.0: "Mayotte",
    178.0: "Congo (Brazzaville)",
    180.0: "Democratic Republic of Congo",
    184.0: "Cook Island/Oceania",
    188.0: "Costa Rica",
    191.0: "Croatia",
    192.0: "Cuba",
    203.0: "Czech Republic",
    204.0: "Benin (formerly Dahomey)",
    208.0: "Denmark",
    212.0: "Dominica",
    214.0: "Dominican Republic",
    218.0: "Ecuador",
    222.0: "El Salvador",
    226.0: "Equatorial Guinea",
    231.0: "Ethiopia",
    232.0: "Eritrea",
    233.0: "Estonia",
    234.0: "Faroe Islands",
    238.0: "Falkland Islands/British Antarctic",
    239.0: "South Georgia and South Sandwich Islands",
    242.0: "Fiji",
    246.0: "Finland",
    250.0: "France",
    254.0: "French Guiana",
    258.0: "French Polynesia/Tahiti",
    260.0: "French Southern and Antarctic Territories",
    262.0: "Djibouti",
    266.0: "Gabon",
    268.0: "Georgia",
    270.0: "Gambia",
    275.0: "Palestine",
    276.0: "Germany",
    288.0: "Ghana",
    292.0: "Gibraltar",
    296.0: "Oceania Islands",
    300.0: "Greece",
    304.0: "Greenland",
    308.0: "Grenada",
    312.0: "Guadeloupe",
    316.0: "Guam",
    320.0: "Guatemala",
    324.0: "Guinea",
    328.0: "Guyana",
    332.0: "Haiti",
    334.0: "Heard Island and McDonald Islands",
    336.0: "Vatican",
    340.0: "Honduras",
    344.0: "Hong Kong",
    348.0: "Hungary",
    352.0: "Iceland",
    356.0: "India",
    360.0: "Indonesia",
    364.0: "Iran",
    368.0: "Iraq",
    372.0: "Ireland",
    376.0: "Israel",
    380.0: "Italy",
    384.0: "Ivory Coast",
    388.0: "Jamaica",
    392.0: "Japan",
    398.0: "Kazakhstan",
    400.0: "Jordan",
    404.0: "Kenya",
    408.0: "Korea, North",
    410.0: "Korea, South Rep",
    414.0: "Kuwait",
    417.0: "Kyrgyzstan",
    418.0: "Laos",
    422.0: "Lebanon",
    426.0: "Lesotho",
    428.0: "Latvia",
    430.0: "Liberia",
    434.0: "Libya",
    438.0: "Liechtenstein",
    440.0: "Lithuania",
    442.0: "Luxembourg",
    446.0: "Macao",
    450.0: "Madagascar",
    454.0: "Malawi",
    458.0: "Malaysia",
    462.0: "Maldives",
    466.0: "Mali",
    470.0: "Malta",
    474.0: "Martinique",
    478.0: "Mauritania",
    480.0: "Mauritius",
    484.0: "Mexico",
    492.0: "Monaco",
    496.0: "Mongolia",
    498.0: "Moldova",
    499.0: "Montenegro",
    500.0: "Montserrat",
    504.0: "Morocco",
    508.0: "Mozambique",
    512.0: "Oman",
    516.0: "Namibia",
    520.0: "Nauru/Oceania",
    524.0: "Nepal",
    528.0: "Netherlands",
    530.0: "Antilles/Curacao",
    533.0: "Aruba",
    540.0: "New Caledonia",
    548.0: "Vanuatu",
    554.0: "New Zealand",
    558.0: "Nicaragua",
    562.0: "Niger",
    566.0: "Nigeria",
    570.0: "Niue Island",
    574.0: "Norfolk Island",
    578.0: "Norway",
    583.0: "Micronesia",
    585.0: "Palau",
    586.0: "Pakistan",
    591.0: "Panama",
    598.0: "Papua New Guinea",
    600.0: "Paraguay",
    604.0: "Peru",
    608.0: "Philippines",
    612.0: "Pitcairn Island/Oceania",
    616.0: "Poland",
    620.0: "Portugal/Portucalense/Portugal",
    621.0: "Madeira/Azores",
    624.0: "Guinea - Bissau",
    626.0: "East Timor",
    630.0: "Puerto Rico",
    634.0: "Qatar",
    638.0: "Reunion Island",
    642.0: "Romania",
    643.0: "Russia",
    646.0: "Rwanda",
    652.0: "St Barthelemy",
    654.0: "Ascension I/St Helena/Trist",
    659.0: "Nevis/St Kitts",
    660.0: "Anguilla",
    662.0: "St Lucia",
    663.0: "St Martin",
    666.0: "St Pierre et Miquelon",
    670.0: "Grenadines/St Vincent",
    674.0: "San Marino",
    678.0: "Sao Tome",
    682.0: "Saudi Arabia",
    686.0: "Senegal",
    688.0: "Serbia",
    690.0: "Seychelles",
    694.0: "Sierra Leone",
    702.0: "Singapore",
    703.0: "Slovakia",
    704.0: "Vietnam",
    705.0: "Slovenia",
    706.0: "Somalia",
    710.0: "South Africa",
    716.0: "Zimbabwe",
    732.0: "Western Sahara",
    736.0: "Sudan",
    740.0: "Dutch Guiana",
    744.0: "Svalbard and Jan Mayen",
    748.0: "Swaziland",
    752.0: "Sweden",
    756.0: "Switzerland",
    760.0: "Syria",
    762.0: "Tajikistan",
    764.0: "Thailand",
    768.0: "Togo",
    772.0: "Tokelau Island/Oceania",
    776.0: "Tonga/Oceania",
    780.0: "Trinidad & Tobago",
    784.0: "United Arab Emirates",
    788.0: "Tunisia",
    792.0: "Turkey",
    795.0: "Turkmenistan",
    796.0: "Turks & Caicos Islands",
    798.0: "Ellice Island/Oceania",
    800.0: "Uganda",
    804.0: "Ukraine",
    807.0: "Macedonia",
    818.0: "Egypt",
    830.0: "British Overseas",
    831.0: "Guernsey - Channel Islands",
    832.0: "Jersey - Channel Islands",
    833.0: "Isle of Man",
    834.0: "Tanzania/Zanzibar",
    840.0: "USA/United States of America",
    850.0: "Virgin Islands (USA)",
    854.0: "Burkina Faso",
    858.0: "Uruguay",
    860.0: "Uzbekistan",
    862.0: "Venezuela",
    876.0: "Wallis and Futuna Islands",
    882.0: "Samoa, Western/Oceania",
    887.0: "Yemen (North & South)",
    894.0: "Zambia",
    901.0: "Southern (Greek) Cyprus",
    902.0: "Turkish Republic of North Cyprus",
    911.0: "Spain/Balearic",
    912.0: "Canary Islands",
    921.0: "England",
    922.0: "Northern Ireland",
    923.0: "Scotland",
    924.0: "Wales",
    926.0: "UK/United Kingdom",
    931.0: "Channel Islands",
    940.0: "Cruise - Europe/Dep",
    941.0: "Cruise - Elsewhere/Departure",
    942.0: "Cruise - Europe/Arrival: UK ship",
    943.0: "Cruise - Europe/Arrival: Foreign ship",
    944.0: "Cruise - Europe/Arrival: Danish ship",
    945.0: "Cruise - Elsewhere/Arrival: UK ship",
    946.0: "Cruise - Elsewhere/Arrival: Foreign ship",
    947.0: "Cruise - Elsewhere/Arrival: Danish ship",
    949.0: "Cruise - Danish ports of arrival and departure",
    950.0: "Short Haul/SOB",
    951.0: "Kosova",
    958.0: "Other Stateless",
    968.0: "Coding Query",
    969.0: "Danish Country of Residence/Birth",
    973.0: "Yugoslavia"
}

In [10]:
Nationality_value_map_22 = {
    0.0: 'Country not disclosed',
    4.0: 'Afghanistan',
    8.0: 'Albania',
    10.0: 'Antarctica',
    12.0: 'Algeria',
    16.0: 'American Samoa/Oceania',
    20.0: 'Andorra',
    24.0: 'Angola',
    28.0: 'Antigua',
    31.0: 'Azerbaijan',
    32.0: 'Argentina',
    36.0: 'Australia',
    40.0: 'Austria',
    44.0: 'Bahamas',
    48.0: 'Bahrain',
    50.0: 'Bangladesh',
    51.0: 'Armenia',
    52.0: 'Barbados',
    56.0: 'Belgium',
    60.0: 'Bermuda',
    64.0: 'Bhutan',
    68.0: 'Bolivia',
    70.0: 'Bosnia Herzegovina',
    72.0: 'Botswana',
    74.0: 'Bouvet Island',
    76.0: 'Brazil',
    84.0: 'Belize',
    86.0: 'British Indian Ocean Territory',
    90.0: 'Solomon Island',
    92.0: 'Virgin Islands (Br)',
    96.0: 'Brunei',
    100.0: 'Bulgaria',
    104.0: 'Myanmar (Burma)',
    108.0: 'Burundi',
    112.0: 'Belarus',
    116.0: 'Cambodia/Kampuchea',
    120.0: 'Cameroon',
    124.0: 'Canada',
    132.0: 'Cape Verde Islands',
    136.0: 'Cayman Islands',
    140.0: 'Central African Rep',
    144.0: 'Sri Lanka',
    148.0: 'Chad',
    152.0: 'Chile',
    156.0: 'China (excl Taiwan)/Tibet',
    158.0: 'Taiwan',
    162.0: 'Christmas Is/Oceania',
    166.0: 'Cocos I/Oceania',
    170.0: 'Colombia',
    174.0: 'Comoros',
    175.0: 'Mayotte',
    178.0: 'Congo (Brazzaville)',
    180.0: 'Democratic Republic of Congo',
    184.0: 'Cook Is/Oceania',
    188.0: 'Costa Rica',
    191.0: 'Croatia',
    192.0: 'Cuba',
    203.0: 'Czech Republic',
    204.0: 'Benin',
    208.0: 'Denmark',
    212.0: 'Dominica',
    214.0: 'Dominican Republic',
    218.0: 'Ecuador',
    222.0: 'El Salvador',
    225.0: 'South Sudan',
    226.0: 'Equatorial Guinea',
    231.0: 'Ethiopia',
    232.0: 'Eritrea',
    233.0: 'Estonia',
    234.0: 'Faroe Islands',
    238.0: 'Falkland Is/British Antarctic',
    239.0: 'South Georgia/South Sandwich Islands',
    242.0: 'Fiji',
    246.0: 'Finland',
    248.0: 'Aland Islands',
    250.0: 'France/Corsica',
    254.0: 'French Guiana',
    258.0: 'French Polynesia/Tahiti',
    260.0: 'French Southern/Antarctic Territories',
    262.0: 'Djibouti',
    266.0: 'Gabon',
    268.0: 'Georgia',
    270.0: 'Gambia',
    275.0: 'Palestine',
    276.0: 'Germany',
    288.0: 'Ghana',
    292.0: 'Gibraltar',
    296.0: 'Oceania Islands',
    300.0: 'Greece/Crete/Rhodes',
    304.0: 'Greenland',
    308.0: 'Grenada',
    312.0: 'Guadeloupe',
    316.0: 'Guam',
    320.0: 'Guatemala',
    324.0: 'Guinea',
    328.0: 'Guyana',
    332.0: 'Haiti',
    334.0: 'Heard & McDonald Islands',
    336.0: 'Vatican',
    340.0: 'Honduras',
    344.0: 'Hong Kong',
    345.0: 'Hong Kong Special',
    348.0: 'Hungary',
    352.0: 'Iceland',
    356.0: 'India',
    360.0: 'Bali/Borneo/Indonesia',
    364.0: 'Iran',
    368.0: 'Iraq',
    372.0: 'Irish Republic',
    376.0: 'Israel',
    380.0: 'Italy/Sardinia',
    384.0: 'Ivory Coast',
    388.0: 'Jamaica',
    392.0: 'Japan',
    398.0: 'Kazakhstan',
    400.0: 'Jordan',
    404.0: 'Kenya',
    408.0: 'North Korea',
    410.0: 'South Korea',
    414.0: 'Kuwait',
    417.0: 'Kyrgyzstan',
    418.0: 'Laos',
    422.0: 'Lebanon',
    426.0: 'Lesotho',
    428.0: 'Latvia',
    430.0: 'Liberia',
    434.0: 'Libya',
    438.0: 'Liechtenstein',
    440.0: 'Lithuania',
    442.0: 'Luxembourg',
    446.0: 'Macao',
    450.0: 'Madagascar',
    454.0: 'Malawi',
    458.0: 'Malaysia',
    462.0: 'Maldives',
    466.0: 'Mali',
    470.0: 'Malta',
    474.0: 'Martinique',
    475.0: 'Curacao',
    477.0: 'Bonaire',
    478.0: 'Mauritania',
    479.0: 'St Maarten',
    480.0: 'Mauritius',
    484.0: 'Mexico',
    492.0: 'Monaco',
    496.0: 'Mongolia',
    498.0: 'Moldova',
    499.0: 'Montenegro',
    500.0: 'Montserrat',
    504.0: 'Morocco',
    508.0: 'Mozambique',
    512.0: 'Oman',
    516.0: 'Namibia',
    520.0: 'Nauru/Oceania',
    524.0: 'Nepal',
    528.0: 'Holland',
    530.0: 'Antilles',
    531.0: 'Curacao',
    533.0: 'Aruba',
    534.0: 'Netherlands Antilles',
    540.0: 'New Caledonia',
    548.0: 'Vanuatu',
    554.0: 'New Zealand',
    558.0: 'Nicaragua',
    562.0: 'Niger',
    566.0: 'Nigeria',
    570.0: 'Niue Island',
    574.0: 'Norfolk Island',
    578.0: 'Norway',
    580.0: 'Mariana Island',
    581.0: 'Pacific Islands',
    583.0: 'Micronesia',
    584.0: 'Marshall Island',
    585.0: 'Palau',
    586.0: 'Pakistan',
    591.0: 'Panama',
    598.0: 'Papua New Guinea',
    600.0: 'Paraguay',
    604.0: 'Peru',
    608.0: 'Philippines',
    612.0: 'Pitcairn Islands',
    616.0: 'Poland',
    620.0: 'Portugal',
    621.0: 'Madeira/Azores',
    624.0: 'Guinea - Bissau',
    626.0: 'East Timor',
    630.0: 'Puerto Rico',
    634.0: 'Qatar',
    638.0: 'Reunion Island',
    642.0: 'Romania',
    643.0: 'Russia',
    646.0: 'Rwanda',
    652.0: 'St Barthelemy',
    654.0: 'Ascension Islands/St Helena/Tristan da Cunha',
    659.0: 'Nevis/St Kitts',
    660.0: 'Anguilla',
    662.0: 'St Lucia',
    663.0: 'St Martin',
    666.0: 'St Pierre et Miquelon',
    670.0: 'Grenadines/St Vincent',
    674.0: 'San Marino',
    678.0: 'Sao Tome',
    682.0: 'Saudi Arabia',
    686.0: 'Senegal',
    688.0: 'Serbia',
    690.0: 'Seychelles',
    694.0: 'Sierra Leone',
    702.0: 'Singapore',
    703.0: 'Slovakia',
    704.0: 'Vietnam',
    705.0: 'Slovenia',
    706.0: 'Somalia',
    710.0: 'South Africa',
    716.0: 'Zimbabwe',
    728.0: 'South Sudan',
    729.0: 'North Sudan',
    740.0: 'Surinam/Dutch Guiana',
    748.0: 'Swaziland',
    752.0: 'Sweden',
    756.0: 'Switzerland',
    760.0: 'Syria',
    762.0: 'Tajikistan',
    764.0: 'Thailand',
    768.0: 'Togo',
    780.0: 'Trinidad & Tobago',
    784.0: 'United Arab Emirates',
    788.0: 'Tunisia',
    792.0: 'Turkey',
    795.0: 'Turkmenistan',
    796.0: 'Turks & Caicos Islands',
    800.0: 'Uganda',
    804.0: 'Ukraine',
    807.0: 'Macedonia',
    818.0: 'Egypt',
    830.0: 'British Overseas Territories',
    831.0: 'Guernsey',
    832.0: 'Jersey',
    833.0: 'Isle Of Man',
    834.0: 'Tanzania',
    840.0: 'USA',
    850.0: 'US Virgin Isles',
    854.0: 'Burkina Faso',
    858.0: 'Uruguay',
    860.0: 'Uzbekistan',
    862.0: 'Venezuela',
    876.0: 'Wallis & Futuna Islands',
    882.0: 'Samoa',
    887.0: 'Yemen (North & South)',
    894.0: 'Zambia',
    901.0: 'South Cyprus',
    902.0: 'North Cyprus',
    911.0: 'Spain',
    912.0: 'Canary Islands',
    926.0: 'UK',
    931.0: 'Channel Islands',
    940.0: 'Cruise - Europe/Departures',
    941.0: 'Cruise - Elsewhere/Departures',
    942.0: 'Cruise - Europe/Arrivals - UK Ship',
    943.0: 'Cruise - Europe/Arrivals - Foreign Ship',
    944.0: 'Cruise - Europe/Arrivals - DK Ship',
    945.0: 'Cruise - Elsewhere/Arrivals - UK Ship',
    946.0: 'Cruise - Elsewhere/Arrivals - Foreign Ship',
    947.0: 'Cruise - Elsewhere/Arrivals - DK Ship',
    949.0: 'Cruise - DK where - Arr & Dep',
    950.0: 'Short Haul',
    951.0: 'Kosova',
    958.0: 'Stateless',
    9999.0: 'Country Not Stated'
}
    

In [11]:
#function to create Nationality_Label column for 2009-2019

def create_nationality_label_column(df):
    df['Nationality'].replace(' ', np.nan, inplace=True)
    df['Nationality'] = pd.to_numeric(df['Nationality'], errors='coerce')
    
    # Create a new column "Purpose_Label" by mapping the values
    df['Nationality_Label'] = df['Nationality'].map(Nationality_value_map_0919)
    df['Nationality_Label'].fillna(("Unknown"), inplace=True)
    df['Nationality'].fillna(-1, inplace=True) #fill unknown values with -1
    
#call this function for df_qcontcust of each year
dataframes = [df_qcontcust_2009, df_qcontcust_2010, df_qcontcust_2011, df_qcontcust_2012,
              df_qcontcust_2013, df_qcontcust_2014, df_qcontcust_2015, df_qcontcust_2016,
              df_qcontcust_2017, df_qcontcust_2018, df_qcontcust_2019]

#iterate over the list of dataframes and apply the function for 2009-2019
for df in dataframes:
    create_nationality_label_column(df)


In [12]:
#create the column for 2022

df_qcontcust_2022['Nationality'].replace(' ', np.nan, inplace=True)
df_qcontcust_2022['Nationality'] = pd.to_numeric(df_qcontcust_2022['Nationality'], errors='coerce')
#Purpose column was not numeric
df_qcontcust_2022['Nationality_Label'] = df_qcontcust_2022['Nationality'].map(Nationality_value_map_22)
df_qcontcust_2022['Nationality_Label'].fillna(("Unknown"), inplace=True)
df_qcontcust_2022['Nationality'].replace('np.nan', -1, inplace=True)


In [36]:
#function to create Stay_Category

def create_stay_category_column(df):
    #the variable names are different across the dataframes so we accomodate this
    stay_column_name = next((col for col in ['Stay', 'stay'] if col in df.columns), None)
    if stay_column_name is not None:
        df[stay_column_name] = pd.to_numeric(df[stay_column_name], errors='coerce')
    
    #we only want to consider stays for less than a year
    #removing outliers in Stay duration, only retaining stays that are less than a year
    df[stay_column_name] = df[stay_column_name].astype(float)
    df[stay_column_name] = df[stay_column_name][df[stay_column_name] <= 365]
    intervals = [1, 3, 13, 27, 90, 180, 365]
    labels = ['1-3 days', '4-13 days', '14-27 days', '1-3 months', '3-6 months', '6-12 months']
    df['Stay_Category'] = pd.cut(df[stay_column_name], bins=intervals, labels=labels)
    
#call this function for df_qcontcust of each year
dataframes = [df_qcontcust_2009, df_qcontcust_2010, df_qcontcust_2011, df_qcontcust_2012,
              df_qcontcust_2013, df_qcontcust_2014, df_qcontcust_2015, df_qcontcust_2016,
              df_qcontcust_2017, df_qcontcust_2018, df_qcontcust_2019, df_qcontcust_2022]

#iterate over the list of dataframes and apply the function for 2009-2019
for df in dataframes:
    create_stay_category_column(df)


In [34]:
# function to create Spend_Category

def create_spend_category_column(df):
    # the variable names are different across the dataframes so we accommodate this
    spend_column_name = next((col for col in ['Spend', 'spend'] if col in df.columns), None)
    if spend_column_name is not None:
        df[spend_column_name] = pd.to_numeric(df[spend_column_name], errors='coerce')

        # we only want to consider expenditure less than 10k as more than that would be outliers
        df[spend_column_name] = df[spend_column_name].astype(float)
        df[spend_column_name] = df[spend_column_name][df[spend_column_name] <= 10000]

        intervals = [0, 250, 500, 1000, 5000, float('inf')]
        labels = ['0-250 GBP', '250-500 GBP', '500-1000 GBP', '1000-5000 GBP', 'more than 5000 GBP']
        df['Spend_Category'] = pd.cut(df[spend_column_name], bins=intervals, labels=labels)

# call this function for df_qcontcust of each year
dataframes = [df_qcontcust_2009, df_qcontcust_2010, df_qcontcust_2011, df_qcontcust_2012,
              df_qcontcust_2013, df_qcontcust_2014, df_qcontcust_2015, df_qcontcust_2016,
              df_qcontcust_2017, df_qcontcust_2018, df_qcontcust_2019, df_qcontcust_2022]

# iterate over the list of dataframes and apply the function for 2009-2019
for df in dataframes:
    create_spend_category_column(df)


In [ ]:
df_qcontcust_2022['Flow'].replace(' ', '-1', inplace=True)
df_qcontcust_2022['Age'].replace(' ', -1, inplace=True)
df_qcontcust_2022['Age'].replace(np.nan, -1, inplace=True)
df_qcontcust_2022['Sex'].replace(' ', np.nan, inplace=True)
df_qcontcust_2022['Spend'].replace(' ', np.nan, inplace=True)
df_qcontcust_2022['Stay'].replace(' ', np.nan, inplace=True)
df_qcontcust_2022['Residence'].replace(' ', np.nan, inplace=True)
df_qcontcust_2022['Flow'].isnull().sum()